In [2]:
! pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.72
    Uninstalling opencv-python-4.7.0.72:
      Successfully uninstalled opencv-python-4.7.0.72


In [3]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [4]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from loader_util.nn.conv import MiniGooglenet
from loader_util.callbacks import TrainingMonitor
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

In [5]:
# script constants
num_epochs = 70
init_lr = 0.001
output_path = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter11_minigooglenet/output"

In [6]:
def poly_decay(epoch):
    max_epochs = num_epochs
    base_lr = init_lr
    power = 1.0

    new_lr = base_lr * (1 - (epoch / float(max_epochs))) ** power
    return new_lr

In [7]:
# loading the data
print(f"[INFO] loading cifar10 data......")
# trainx => 50K x 32 x 32 x 3
(trainx, trainy), (testx, testy) = cifar10.load_data()
trainx = trainx.astype("float")
testx = testx.astype("float")

# apply mean subtraction to data
mean = np.mean(trainx, axis=0)
trainx -= mean
testx -= mean

# encode labels
lb = LabelBinarizer()
trainy_encoded = lb.fit_transform(trainy)
testy_encoded = lb.transform(testy)

[INFO] loading cifar10 data......
170498071/170498071 [==============================] - 6s 0us/step


In [8]:
# data augmentor to combat overfitting
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [10]:
# construct the callbacks
fig_path = os.path.sep.join([output_path, f"{os.getpid()}.png"])
json_path = os.path.sep.join([output_path, f"{os.getpid()}.json"])
call_backs = [TrainingMonitor(figPath=fig_path, jsonPath=json_path),
              LearningRateScheduler(poly_decay)]

In [11]:
# init the opt and model
print(f"[INFO]compiling the opt and model......")
opt = SGD(learning_rate=init_lr, momentum=0.9)
model = MiniGooglenet.build(32, 32, 3, classes=10)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])


[INFO]compiling the opt and model......


In [12]:
print(f"[INFO] training the network......")
model.fit_generator(aug.flow(trainx, trainy_encoded, batch_size=64),
                    validation_data=(testx, testy_encoded),
                    steps_per_epoch=len(trainx) // 64,
                    epochs=num_epochs,
                    callbacks=call_backs)

print(f"[INFO] saving the model......")
model_path = os.path.sep.join([output_path, f"saved_model.h5"])
model.save(model_path)

[INFO] training the network......
Epoch 1/70


<ipython-input-12-12b1f5f72b60>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(aug.flow(trainx, trainy_encoded, batch_size=64),


781/781 [==============================] - 59s 56ms/step - loss: 1.7056 - accuracy: 0.3665 - val_loss: 1.5600 - val_accuracy: 0.4620 - lr: 0.0010
Epoch 2/70
781/781 [==============================] - 44s 56ms/step - loss: 1.3425 - accuracy: 0.5132 - val_loss: 1.2945 - val_accuracy: 0.5501 - lr: 9.8571e-04
Epoch 3/70
781/781 [==============================] - 43s 55ms/step - loss: 1.1859 - accuracy: 0.5756 - val_loss: 1.4721 - val_accuracy: 0.5289 - lr: 9.7143e-04
Epoch 4/70
781/781 [==============================] - 44s 56ms/step - loss: 1.0829 - accuracy: 0.6134 - val_loss: 1.1639 - val_accuracy: 0.5795 - lr: 9.5714e-04
Epoch 5/70
781/781 [==============================] - 44s 56ms/step - loss: 0.9976 - accuracy: 0.6487 - val_loss: 1.0533 - val_accuracy: 0.6326 - lr: 9.4286e-04
Epoch 6/70
781/781 [==============================] - 43s 55ms/step - loss: 0.9344 - accuracy: 0.6717 - val_loss: 0.8457 - val_accuracy: 0.7003 - lr: 9.2857e-04
Epoch 7/70
781/781 [=============================